In [1]:
#imporitng all the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer
import numpy_financial as npf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Bondora_raw.csv')
df.shape

(134529, 112)

In [3]:
# Viewing the entire imported dataset
pd.set_option('display.max_columns', 120)
df.head()

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,UserName,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,ContractEndDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,DateOfBirth,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,NrOfDependants,EmploymentStatus,EmploymentDurationCurrentEmployer,EmploymentPosition,WorkExperience,OccupationArea,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeFromPension,IncomeFromFamilyAllowance,IncomeFromSocialWelfare,IncomeFromLeavePay,IncomeFromChildSupport,IncomeOther,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedPrincipalTillDate,PlannedInterestTillDate,LastPaymentOn,CurrentDebtDaysPrimary,DebtOccuredOn,CurrentDebtDaysSecondary,DebtOccuredOnForSecondary,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,DefaultDate,PrincipalOverdueBySchedule,PlannedPrincipalPostDefault,PlannedInterestPostDefault,EAD1,EAD2,PrincipalRecovery,InterestRecovery,RecoveryStage,StageActiveSince,ModelVersion,Rating,EL_V0,Rating_V0,EL_V1,Rating_V1,Rating_V2,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEsEquifaxRisk,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
0,2020-01-27,F0660C80-83F3-4A97-8DA0-9C250112D6EC,659,2009-06-11 16:40:39,2009-06-11 16:40:39,0,0,115.0410,KARU,True,2009-06-11 16:40:39,2009-06-16,2010-07-06,2009-07-27,2010-06-25,2010-06-25,17,5,2.0,1,61,1947-11-26,1.0,EE,319.5582,115.0408,30.0,12,NaN,HARJU,TALLINN,7,3.0,1.0,0,3.0,UpTo3Years,klienditeenindaja,MoreThan25Years,7.0,NaN,6000.0,0.0,0.0,0.0,0.0,0.0,4500.0,10500.0,0,0.0,0,0.0,0.0,25,True,1800.0000,319.0800,2010-07-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,91-120,NaN,NaN,NaN,NaN,115.0408,20.4222,0.0,0.0,0.00,0.00,1,83.0852,0.0000,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,2020-01-27,978BB85B-1C69-4D51-8447-9C240104A3A2,654,2009-06-10 15:48:57,2009-06-10 15:48:57,0,0,140.6057,koort681,False,2009-06-10 15:48:57,2009-06-15,2009-07-07,2009-07-15,2009-07-15,2009-07-15,20,4,2.0,1,48,1960-11-05,1.0,EE,191.7349,140.6057,25.0,1,NaN,IDA-VIRU,KOHTLA-JARVE,7,5.0,4.0,0,3.0,MoreThan5Years,Õppealajuhataja lasteaias,MoreThan25Years,16.0,NaN,8300.0,0.0,0.0,0.0,0.0,0.0,2500.0,10800.0,0,0.0,0,0.0,0.0,15,True,2200.0000,45.8300,2009-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,NaN,NaN,NaN,NaN,NaN,140.6057,2.0227,0.0,0.0,0.00,0.00,2,255.6467,258.6256,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2,2020-01-27,EA44027E-7FA7-4BB2-846D-9C1F013C8A22,641,2009-06-05 19:12:29,2009-06-05 19:12:29,0,0,319.5580,0ie,True,2009-06-05 19:12:29,2009-06-15,NaN,2009-07-27,2011-02-25,2014-05-13,20,6,2.0,1,58,1950-11-13,1.0,EE,319.5582,319.5409,25.0,20,NaN,PÄRNU,PÄRNU,2,4.0,1.0,0,3.0,UpTo4Years,teenindaja,MoreThan25Years,9.0,NaN,5000.0,0.0,0.0,0.0,0.0,0.0,2000.0,7000.0,0,0.0,0,0.0,0.0,25,True,319.5409,197.2926,2012-10-01,2813.0,2012-05-14,2935.0,2012-01-13,NaN,NaN,NaN,NaN,2012-07-16,116.35,236.97,38.24,279.5049,270.7323,163.1549,50.99,NaN,2016-03-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Late,True,180+,180+,NaN,NaN,NaN,NaN,203.1909,59.7626,0.0,0.0,116.35,414.07,0,0.0000,

In [4]:
df['Amount'].isnull().sum()

0

In [5]:
df['LoanDuration'].isnull().sum()

0

In [6]:
df['Interest'].isnull().sum()

0

In [7]:
df['LiabilitiesTotal'].isnull().sum()

0

In [8]:
df['AppliedAmount'].isnull().sum()

0

In [9]:
df['IncomeTotal'].isnull().sum()

0

###### LoanTenure and LoanDuration are same

# Equated Monthly Installments (EMI)


In [10]:
import pandas as pd

def calculate_emi(row):
    # Extract the required values from the row
    amount = row['Amount']
    interest_rate = row['Interest']
    loan_tenure = row['LoanDuration']
    
    # Calculate result_1 = P * r * (1 + r) ** n
    result_1 = amount * interest_rate * (1 + interest_rate) ** loan_tenure
    
    # Calculate result_2 = (1 + r) ** n - 1
    result_2 = (1 + interest_rate) ** loan_tenure - 1
    
    # Calculate EMI using the flat rate method formula
    emi = result_1 / result_2
    
    return emi

# Example usage:
# Assuming your data is stored in a DataFrame called 'df'
# Apply the calculate_emi function to each row to get the EMI
df['EMI'] = df.apply(calculate_emi, axis=1)

df['EMI']

0          3451.2240
1          3655.7482
2          7988.5225
3          2588.4225
4          9586.3080
             ...    
134524    93030.0000
134525    54750.0000
134526    62075.0000
134527    53220.0000
134528    91500.0000
Name: EMI, Length: 134529, dtype: float64

# Eligible Loan Amount (ELA)

In [11]:
import pandas as pd

def calculate_ela(row):
    # Extract the required values from the row
    applied_amount = row['AppliedAmount']
    interest_rate = row['Interest']
    loan_tenure = row['LoanDuration']
    income_total = row['IncomeTotal']
    liabilities_total = row['LiabilitiesTotal']
    
    # Calculate Total Payment Due
    total_payment_due = applied_amount + (applied_amount * interest_rate * loan_tenure)
    
    # Calculate Max allowable amount
    max_allowable_amount = (income_total - liabilities_total) * 0.30
    
    # Calculate ELA based on the conditions
    if total_payment_due <= max_allowable_amount:
        ela = applied_amount
    else:
        ela = max_allowable_amount
        
    return ela

# Assuming your data is stored in a DataFrame called 'df'
# Apply the calculate_ela function to each row to get the ELA
df['ELA'] = df.apply(calculate_ela, axis=1)
df['ELA']

0         3150.000
1         3240.000
2         2100.000
3         3480.000
4         2040.000
            ...   
134524     270.000
134525     146.262
134526     345.000
134527     -57.615
134528      -9.309
Name: ELA, Length: 134529, dtype: float64

# Preferred Return on Investment(PROI)

In [12]:
import pandas as pd

def calculate_roi(interest_amount, loan_amount):
    return (interest_amount / loan_amount) * 100

def calculate_proi(df):
    # Step 1: Calculate ROI for all rows in the dataset
    df['ROI'] = calculate_roi(df['Interest'], df['AppliedAmount'])
    
    # Step 2: Set PROI as the median of all ROI values in the dataset (anchor value)
    proi = df['ROI'].median()
    
    # Step 3: Check AppliedAmount
    df.loc[(df['AppliedAmount'] >= 850) & (df['AppliedAmount'] <= 1175), 'PROI'] = proi - 5
    df.loc[df['AppliedAmount'] >= 2000, 'PROI'] = proi + 5
    
    # Step 4: Check LoanTenure
    df.loc[df['LoanDuration'] <= 19, 'PROI'] = proi - 5
    df.loc[df['LoanDuration'] > 25, 'PROI'] = proi + 5
    
    # Step 5: Check IncomeTotal
    df.loc[df['IncomeTotal'] <= 1000, 'PROI'] = proi - 5
    
    # Step 6: Check DebtToIncome
    df.loc[df['DebtToIncome'] == 0, 'PROI'] = proi - 5
    df.loc[df['DebtToIncome'] != 0, 'PROI'] = proi + 5
    
    # Step 7: Return final PROI
    return df['PROI']

# Example usage:
# Assuming your data is stored in a DataFrame called 'df'
# Calculate the PROI for the dataset
df['PROI'] = calculate_proi(df)
df['PROI']

0        -3.429647
1        -3.429647
2        -3.429647
3        -3.429647
4        -3.429647
            ...   
134524    6.570353
134525    6.570353
134526    6.570353
134527    6.570353
134528    6.570353
Name: PROI, Length: 134529, dtype: float64

In [13]:
import pandas as pd

column_name_mapping = {
    'EMI': 'EquatedMonthlyInstallments',
    'ELA': 'EligibleLoanAmount',
    'ROI': 'ReturnOnInvestment',
    'PROI': 'PreferredReturnonInvestment'
    
}


df.rename(columns=column_name_mapping, inplace=True)



In [14]:
df.head()

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,UserName,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,ContractEndDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,DateOfBirth,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,NrOfDependants,EmploymentStatus,EmploymentDurationCurrentEmployer,EmploymentPosition,WorkExperience,OccupationArea,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeFromPension,IncomeFromFamilyAllowance,IncomeFromSocialWelfare,IncomeFromLeavePay,IncomeFromChildSupport,IncomeOther,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedPrincipalTillDate,PlannedInterestTillDate,LastPaymentOn,CurrentDebtDaysPrimary,DebtOccuredOn,CurrentDebtDaysSecondary,DebtOccuredOnForSecondary,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,DefaultDate,PrincipalOverdueBySchedule,PlannedPrincipalPostDefault,PlannedInterestPostDefault,EAD1,EAD2,PrincipalRecovery,InterestRecovery,RecoveryStage,StageActiveSince,ModelVersion,Rating,EL_V0,Rating_V0,EL_V1,Rating_V1,Rating_V2,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEsEquifaxRisk,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory,EquatedMonthlyInstallments,EligibleLoanAmount,ReturnOnInvestment,PreferredReturnonInvestment
0,2020-01-27,F0660C80-83F3-4A97-8DA0-9C250112D6EC,659,2009-06-11 16:40:39,2009-06-11 16:40:39,0,0,115.0410,KARU,True,2009-06-11 16:40:39,2009-06-16,2010-07-06,2009-07-27,2010-06-25,2010-06-25,17,5,2.0,1,61,1947-11-26,1.0,EE,319.5582,115.0408,30.0,12,NaN,HARJU,TALLINN,7,3.0,1.0,0,3.0,UpTo3Years,klienditeenindaja,MoreThan25Years,7.0,NaN,6000.0,0.0,0.0,0.0,0.0,0.0,4500.0,10500.0,0,0.0,0,0.0,0.0,25,True,1800.0000,319.0800,2010-07-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,91-120,NaN,NaN,NaN,NaN,115.0408,20.4222,0.0,0.0,0.00,0.00,1,83.0852,0.0000,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,3451.2240,3150.0,9.387961,-3.429647
1,2020-01-27,978BB85B-1C69-4D51-8447-9C240104A3A2,654,2009-06-10 15:48:57,2009-06-10 15:48:57,0,0,140.6057,koort681,False,2009-06-10 15:48:57,2009-06-15,2009-07-07,2009-07-15,2009-07-15,2009-07-15,20,4,2.0,1,48,1960-11-05,1.0,EE,191.7349,140.6057,25.0,1,NaN,IDA-VIRU,KOHTLA-JARVE,7,5.0,4.0,0,3.0,MoreThan5Years,Õppealajuhataja lasteaias,MoreThan25Years,16.0,NaN,8300.0,0.0,0.0,0.0,0.0,0.0,2500.0,10800.0,0,0.0,0,0.0,0.0,15,True,2200.0000,45.8300,2009-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,NaN,NaN,NaN,NaN,NaN,140.6057,2.0227,0.0,0.0,0.00,0.00,2,255.6467,258.6256,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,3655.7482,3240.0,13.038836,-3.429647
2,2020-01-27,EA44027E-7FA7-4BB2-846D-9C1F013C8A22,641,2009-06-05 19:12:29,2009-06-05 19:12:29,0,0,319.5580,0ie,True,2009-06-05 19:12:29,2009-06-15,NaN,2009-07-27,2011-02-25,2014-05-13,20,6,2.0,1,58,1950-11-13,1.0,EE,319.5582,319.5409,25.0,20,NaN,PÄRNU,PÄRNU,2,4.0,1.0,0,3.0,UpTo4Years,teenindaja,MoreThan25Years,9.0,NaN,5000.0,0.0,0.0,0.0,0.0,0.0,2000.0,7000.0,0,0.0,0,0.0,0.0,25,True,319.5409,197.2926,2012-10-01,2813.0,2012-05-14,2935.0,2012-01-13,NaN,NaN,NaN,NaN,2012-07-16,116.35,236.97,38.24,279

In [15]:
for column in df.columns:
    print(column)

ReportAsOfEOD
LoanId
LoanNumber
ListedOnUTC
BiddingStartedOn
BidsPortfolioManager
BidsApi
BidsManual
UserName
NewCreditCustomer
LoanApplicationStartedDate
LoanDate
ContractEndDate
FirstPaymentDate
MaturityDate_Original
MaturityDate_Last
ApplicationSignedHour
ApplicationSignedWeekday
VerificationType
LanguageCode
Age
DateOfBirth
Gender
Country
AppliedAmount
Amount
Interest
LoanDuration
MonthlyPayment
County
City
UseOfLoan
Education
MaritalStatus
NrOfDependants
EmploymentStatus
EmploymentDurationCurrentEmployer
EmploymentPosition
WorkExperience
OccupationArea
HomeOwnershipType
IncomeFromPrincipalEmployer
IncomeFromPension
IncomeFromFamilyAllowance
IncomeFromSocialWelfare
IncomeFromLeavePay
IncomeFromChildSupport
IncomeOther
IncomeTotal
ExistingLiabilities
LiabilitiesTotal
RefinanceLiabilities
DebtToIncome
FreeCash
MonthlyPaymentDay
ActiveScheduleFirstPaymentReached
PlannedPrincipalTillDate
PlannedInterestTillDate
LastPaymentOn
CurrentDebtDaysPrimary
DebtOccuredOn
CurrentDebtDaysSecondary

In [16]:
df.shape

(134529, 116)

# Data Preprocessing